<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Emotion-Identification2/custom_Model_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import imghdr
import tensorflow as tf
from PIL import Image
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.applications import VGG16, ResNet50V2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import plot_model

In [ ]:
# Load preprocessed data from .npy files
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")

# Define the list of acceptable image extensions (for reference, not used here)
image_exts = ['jpeg', 'jpg', 'png']

# Function to validate and preprocess images (if needed)
def validate_and_preprocess(image):
    """
    Validate and preprocess an image (if needed).
    This function can be used to perform additional checks or transformations.
    """
    # Example: Check if the image is valid (not empty)
    if image is None or image.size == 0:
        print("Invalid image detected.")
        return None

    # Example: Convert to grayscale if not already
    if len(image.shape) == 3 and image.shape[2] == 3:  # Check if RGB
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    return image

# Iterate through the preprocessed images and labels
for i, (image, label) in enumerate(zip(X_train, y_train)):
    try:
        # Validate and preprocess the image (if needed)
        processed_image = validate_and_preprocess(image)

        if processed_image is None:
            print(f"Skipping invalid image at index {i}.")
            continue

        # Update the image in the dataset (if preprocessing was applied)
        X_train[i] = processed_image

    except Exception as e:
        # Handle any exceptions that occur during processing
        print(f"Issue with image at index {i}. Error: {e}")

# Save the updated dataset (if any changes were made)
np.save("X_train_validated.npy", X_train)
np.save("y_train_validated.npy", y_train)

print("Validation and preprocessing complete!")

In [ ]:
import os
import numpy as np
import pandas as pd

# Define a function to count the number of samples in preprocessed .npy files
def count_samples_in_npy(data_dir, set_name):
    """
    Count the number of samples in preprocessed .npy files for each class.

    Args:
        data_dir (str): Directory containing the .npy files.
        set_name (str): Name of the dataset (e.g., 'train' or 'test').

    Returns:
        pd.DataFrame: A DataFrame with the counts of samples for each class.
    """
    # Initialize an empty dictionary to hold the count of samples for each class
    counts = {}

    # Load the preprocessed data
    X = np.load(os.path.join(data_dir, f"X_{set_name}.npy"))
    y = np.load(os.path.join(data_dir, f"y_{set_name}.npy"))

    # Get unique classes and their counts
    unique_classes, class_counts = np.unique(y, return_counts=True)

    # Map class indices to class names (if you have a label encoder or class names)
    # Assuming you have a list of class names in the same order as the encoded labels
    class_names = ["anger", "happy", "sad", "neutral", "surprise", "fear", "disgust"]

    # Populate the counts dictionary
    for class_idx, count in zip(unique_classes, class_counts):
        class_name = class_names[class_idx]  # Map class index to class name
        counts[class_name] = count

    # Convert the counts dictionary to a DataFrame for easy viewing and analysis
    df = pd.DataFrame(counts, index=[set_name])
    return df

# Paths to the directories containing the preprocessed .npy files
train_data_dir = 'C:/Users/LENOVO/PyCharmMiscProject' # Directory containing X_train.npy and y_train.npy
test_data_dir = 'C:/Users/LENOVO/PyCharmMiscProject'# Directory containing X_test.npy and y_test.npy

# Count the samples in the training dataset and print the result
train_count = count_samples_in_npy(train_data_dir, 'train')
print(train_count)

# Count the samples in the testing dataset and print the result
test_count = count_samples_in_npy(test_data_dir, 'test')
print(test_count)